

# Basic AequilibraE use

In this example, we grab a pre-existing model and do the following:

* Add new fields to the network
* Re-compute fields in the network
* Export existing matrices to OMX
* Perform Skimming
* Perform Assignment


## Running on Google Colab

Press here to open this notebook in Google Colab <a href="https://colab.research.google.com/github/outerl/AequilibraE-demo/blob/main/basic_aequilibrae_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Once in Colab, uncomment the cell below and run it.

**It will restart your Python environment to load all packages correctly**, so it is not possible just run all cells

In [1]:
# !apt-get update && apt-get install libsqlite3-mod-spatialite
# !apt-get install -y libspatialite-dev
# !pip install "numpy>1.22,<1.24" --upgrade
# !pip install aequilibrae matplotlib shapely openmatrix

# Before we begin
Let's make a copy of the model so we can make the changes we want without overwriting the original model

If you have are running this notebook on Google Colab and want to save your model into your Google Drive, you should:

1. Un-comment and run the two cells below
2. Accept the terms and conditions when asking to connect Colab to Google Drive.
3. Skip the following cell, where we copy the model around

In [2]:
# #Just in case you wabt to use Google colab
# from google.colab import drive
# drive.mount("/content/gdrive")

In [ ]:
# !wget https://github.com/outerl/AequilibraE-demo/releases/download/Freeworld/LongAn_base_model.zip
# import zipfile
# with zipfile.ZipFile('LongAn_base_model.zip',"r") as zip_ref:
#     zip_ref.extractall(".")

# # You probably want to move this into your google drive, but we will leave in iun the temporary folder of the VM running the model
# model_path = "./LongAn_base_model"

If not using Colab, you will need to download the [MODEL](https://github.com/outerl/AequilibraE-demo/releases/download/Freeworld/LongAn_base_model.zip), unzip it and set the folders below accordingly

## Let's do some modeling

In [4]:
# Imports
from os.path import join
from aequilibrae import Project

In [6]:
# We open a project
project = Project.from_path(model_path)

## Model stats

In [7]:
print(f"Links: {project.network.count_links():,}")
print(f"Nodes: {project.network.count_nodes():,}")
print(f"Zones: {project.network.count_centroids():,}")

Links: 384,122
Nodes: 369,872
Zones: 698


Let's see which fields we have in our links layer

In [8]:
project.network.links.fields.all_fields()

['a_node',
 'b_node',
 'direction',
 'distance',
 'link_id',
 'link_type',
 'modes',
 'name',
 'osm_id',
 'lanes',
 'capacity',
 'geometry',
 'speed']

We are missing travel time. Let's add it

In [9]:
fields = project.network.links.fields
fields.add("ff_ttime_ab", description="AB Free flow travel time")
fields.add("ff_ttime_ba", description="BA Free flow travel time")
fields.save()

In [10]:
project.network.links.fields.all_fields()

['a_node',
 'b_node',
 'capacity',
 'direction',
 'distance',
 'geometry',
 'lanes',
 'link_id',
 'link_type',
 'modes',
 'name',
 'osm_id',
 'speed',
 'ff_ttime']

#### Let's use some SQL to compute these fields

Speeds are in KM/h and distances are ALWAYS in meters in AequilibraE


In [11]:
%%time
sql = """Update links set ff_ttime_ab=(distance/1000)/speed_ab * 60, 
                          ff_ttime_ba=(distance/1000)/speed_ab * 60"""
project.conn.execute(sql)
project.conn.commit()

CPU times: total: 734 ms
Wall time: 2.91 s


## Computes Graph



In [12]:
%%time
project.network.build_graphs(modes=["c"])

CPU times: total: 1.58 s
Wall time: 5.73 s


In [13]:
graph = project.network.graphs["c"]
graph.set_graph("ff_ttime")
graph.set_skimming(["distance", "ff_ttime"])

## Skimming

In [14]:
from aequilibrae.paths import NetworkSkimming

In [15]:
%%time
skimmer = NetworkSkimming(graph)
skimmer.execute()

CPU times: total: 10.1 s
Wall time: 824 ms


In [16]:
skimmer.results.skims.distance

array([[0.00000000e+00, 2.08210301e+05, 1.96844618e+06, ...,
        2.30752212e+05, 2.31110720e+05, 2.26458067e+05],
       [2.47165989e+05, 0.00000000e+00, 2.04366321e+06, ...,
        3.05969242e+05, 3.06327750e+05, 3.01675097e+05],
       [1.96299604e+06, 2.03601550e+06, 0.00000000e+00, ...,
        1.75525994e+06, 1.75561845e+06, 1.75709952e+06],
       ...,
       [2.29378467e+05, 3.02397928e+05, 1.75985749e+06, ...,
        0.00000000e+00, 1.54895959e+03, 3.03003335e+03],
       [2.29626108e+05, 3.02645569e+05, 1.76010513e+06, ...,
        1.43809163e+03, 0.00000000e+00, 1.86561573e+03],
       [2.25019026e+05, 2.98038487e+05, 1.76174415e+06, ...,
        3.07710875e+03, 1.89506487e+03, 0.00000000e+00]])

In [17]:
skimmer.results.skims.ff_ttime

array([[0.00000000e+00, 3.54299335e+02, 2.48121136e+03, ...,
        2.86413631e+02, 2.86151490e+02, 2.88266825e+02],
       [3.53429163e+02, 0.00000000e+00, 2.58233037e+03, ...,
        3.87532636e+02, 3.87270495e+02, 3.89385830e+02],
       [2.50353592e+03, 2.60859558e+03, 0.00000000e+00, ...,
        2.25222599e+03, 2.25196385e+03, 2.25514854e+03],
       ...,
       [2.78741510e+02, 3.83801176e+02, 2.22291488e+03, ...,
        0.00000000e+00, 2.64542158e+00, 5.83011813e+00],
       [2.78311411e+02, 3.83371077e+02, 2.22248479e+03, ...,
        2.47746350e+00, 0.00000000e+00, 3.70308397e+00],
       [2.81720163e+02, 3.86779829e+02, 2.22591373e+03, ...,
        5.90640663e+00, 3.74153842e+00, 0.00000000e+00]])

We can save this matrix to disk and add to the model

In [18]:
skimmer.save_to_project("skim_matrix")

## Matrices

In [19]:
project.matrices.list()

,name,file_name,cores,procedure,procedure_id,timestamp,description,status
0,base_demand,base_demand.aem,9,None,None,2023-05-30 03:52:33,None,
1,skim_matrix,skim_matrix.omx,2,Network skimming,ed3807a0df5e4c18b23c98ed58ccc08b,2023-05-30 21:44:30.604460,None,


In [20]:
mat = project.matrices.get_matrix("base_demand")
mat.names

['red_cars_AM',
 'blue_cars_AM',
 'Trucks_AM',
 'red_cars_MD',
 'blue_cars_MD',
 'Trucks_MD',
 'red_cars_PM',
 'blue_cars_PM',
 'Trucks_PM']

Let's export only the matrices for the peak period

In [21]:
mat.export(join(project.matrices.fldr, "peak_demand.omx"), cores=['red_cars_AM', 'blue_cars_AM', 'Trucks_AM', 'red_cars_PM', 'blue_cars_PM', 'Trucks_PM'])

In [22]:
project.matrices.update_database()
project.matrices.list()

,name,file_name,cores,procedure,procedure_id,timestamp,description,status
0,base_demand,base_demand.aem,9,None,None,2023-05-30 03:52:33,None,
1,skim_matrix,skim_matrix.omx,2,Network skimming,ed3807a0df5e4c18b23c98ed58ccc08b,2023-05-30 21:44:30.604460,None,
2,peak_demand_omx,peak_demand.omx,6,None,None,2023-05-30 11:44:31,None,


## Assignment

Let's assign only a few of the matrices we exported

In [23]:
demand_cars = project.matrices.get_matrix("peak_demand_omx")
demand_cars.computational_view(['red_cars_AM', 'blue_cars_AM'])

In [24]:
demand_trucks = project.matrices.get_matrix("peak_demand_omx")
demand_trucks.computational_view(['Trucks_AM'])

In [25]:
from aequilibrae.paths import TrafficAssignment, TrafficClass

In [26]:

# Create the assignment class
car_class = TrafficClass(name="car", graph=graph, matrix=demand_cars)
truck_class = TrafficClass(name="truck", graph=graph, matrix=demand_trucks)

truck_class.set_pce(1.8)
# truck_class.set_select_links()
# truck_class.set_vot()
# truck_class.set_fixed_cost()

In [27]:

assig = TrafficAssignment()

# The first thing to do is to add at list of traffic classes to be assigned
assig.add_class(car_class)
assig.add_class(truck_class)

# We set these parameters only after adding one class to the assignment
assig.set_vdf("BPR")  # This is not case-sensitive

# Then we set the volume delay function
assig.set_vdf_parameters({"alpha": 0.15, "beta": 4.0})  # And its parameters

assig.set_capacity_field("capacity")  # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("ff_ttime")

# And the algorithm we want to use to assign
assig.set_algorithm("bfw")

# Since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 5
assig.rgap_target = 0.0001

assig.execute()  # we then execute the assignment

In [28]:
assig.report()

,iteration,rgap,alpha,warnings,beta0,beta1,beta2
0,1,inf,1.000000,,1.00000,0.00000,0.0
1,2,0.551868,0.420385,,1.00000,0.00000,0.0
2,3,0.150826,0.184132,,1.00000,0.00000,0.0
3,4,0.037613,0.073601,,1.00000,0.00000,0.0
4,5,0.030225,0.085541,,0.79083,0.20917,0.0


In [29]:
assig.results().max()

red_cars_AM_ab        3402.933333
red_cars_AM_ba        3413.275000
red_cars_AM_tot       6816.208333
blue_cars_AM_ab        880.941667
blue_cars_AM_ba        888.533333
blue_cars_AM_tot      1769.475000
Trucks_AM_ab             9.780088
Trucks_AM_ba            12.630000
Trucks_AM_tot           20.243333
Congested_Time_AB       65.107927
Congested_Time_BA       65.107927
Congested_Time_Max      65.107927
Delay_factor_AB         53.783730
Delay_factor_BA         53.783730
Delay_factor_Max        53.783730
VOC_AB                   4.331140
VOC_BA                   4.331140
VOC_max                  4.331140
PCE_AB                4297.579000
PCE_BA                4324.542333
PCE_tot               8622.121333
dtype: float64

In [30]:
assig.save_results("tutorial")